In [1]:
# Загружаем библиотеки
import nltk # Обработка естественного языка
import numpy as np # Математика
from sklearn.model_selection import train_test_split # Разделение выборки на обучающую и тестовую
from collections import Counter # Подсчёт вхождений
from collections import defaultdict # Словарь со значениями по умолчанию
from keras.utils.np_utils import to_categorical # Унитарное кодирование

# Библиотеки для нейросети
import keras
from keras import layers as L

#### Загрузка и обработка занных

In [2]:
# Модули для nltk
nltk.download('brown') # Корпус английского языка, классифицированный по стилям
nltk.download('universal_tagset') # Универсальный набор тегов

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Pavel\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Pavel\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [3]:
# Берём тренировочный датасет из nltk
data = nltk.corpus.brown.tagged_sents(tagset = 'universal') # Заранее размеченный набор слов из общей категории 
all_tags = ['#EOS#','#UNK#','ADV', 'NOUN', 'ADP', 'PRON', 'DET', '.', 'PRT', 'VERB', 'X', 'NUM', 'CONJ', 'ADJ'] # Тэги

# В ячейках матрицы лежат отдельны предложения, они состоят из отдельных слов с тегами
data = np.array([[(word.lower(), tag) for word, tag in sentence] for sentence in data], dtype = object)

In [4]:
# Разделение выборки на обучающую и тестовую
train_data, test_data = train_test_split(data, test_size = 0.25, random_state = 42)

In [5]:
# Подсчёт частотности слов
word_counts = Counter()
for sentence in data:
    words, tags = zip(*sentence)
    word_counts.update(words)

# Берём 10000 наиболее встречаемых слов и теги '#EOS#', '#UNK#'    
all_words = ['#EOS#', '#UNK#'] + list(list(zip(*word_counts.most_common(10000)))[0])

# Доля используемых слов от общего количества слов в словаре
print("Coverage = %.5f" % (float(sum(word_counts[w] for w in all_words)) / sum(word_counts.values())))

Coverage = 0.92876


In [6]:
# Присваиваем поряковые номера словам и тегам
word_to_id = defaultdict(lambda:1, { word: i for i, word in enumerate(all_words) })
tag_to_id = { tag: i for i, tag in enumerate(all_tags)}

In [7]:
# Функция преобразования слов и тегов в матрицу
def to_matrix(lines, token_to_id, max_len = None, pad = 0, dtype = 'int32', time_major = False):
    max_len = max_len or max(map(len, lines))
    matrix = np.empty([len(lines), max_len], dtype)
    matrix.fill(pad)

    for i in range(len(lines)):
        line_ix = list(map(token_to_id.__getitem__,lines[i]))[:max_len]
        matrix[i,:len(line_ix)] = line_ix

    return matrix.T if time_major else matrix

In [8]:
# Посмотрим несколько элементов матрицы
batch_words, batch_tags = zip(*[zip(*sentence) for sentence in data[-3:]])

print("Word ids:")
print(to_matrix(batch_words, word_to_id))
print("Tag ids:")
print(to_matrix(batch_tags, tag_to_id))

Word ids:
[[   2 3057    5    2 2238 1334 4238 2454    3    6   19   26 1070   69
     8 2088    6    3    1    3  266   65  342    2    1    3    2  315
     1    9   87  216 3322   69 1558    4    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]
 [  45   12    8  511 8419    6   60 3246   39    2    1    1    3    2
   845    1    3    1    3   10 9910    2    1 3470    9   43    1    1
     3    6    2 1046  385   73 4562    3    9    2    1    1 3250    3
    12   10    2  861 5240   12    8 8936  121    1    4]
 [  33   64   26   12  445    7 7346    9    8 3337    3    1 2811    3
     2  463  572    2    1    1 1649   12    1    4    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]
Tag ids:
[[ 6  3  4  6  3  3  9  9  7 12  4  5  9  4  6  3 12  7  9  7  9  8  4  6
   3  7  6 13  3  4  6  3  9  4  3  7  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0

#### Модель рекуррентной нейросети

In [9]:
# Нейросеть последовательной архитектуры
model = keras.models.Sequential()
# Входной слой
model.add(L.InputLayer([None], dtype = 'int32'))
# Слой сжатия, количество входов равно количеству всех слов, выходов 50
model.add(L.Embedding(len(all_words), 50))
# Слой полносвязной рекррентной нейросети
model.add(L.SimpleRNN(64, return_sequences = True))

# Выходной слой размерностью равной количеству всех слов выдаёт веростности для тегов
stepwise_dense = L.Dense(len(all_tags), activation = 'softmax')
stepwise_dense = L.TimeDistributed(stepwise_dense)
model.add(stepwise_dense)

In [10]:
BATCH_SIZE=32
def generate_batches(sentences,batch_size=BATCH_SIZE,max_len=None,pad=0):
    assert isinstance(sentences,np.ndarray),"Make sure sentences is q numpy array"
    
    while True:
        indices = np.random.permutation(np.arange(len(sentences)))
        for start in range(0,len(indices)-1,batch_size):
            batch_indices = indices[start:start+batch_size]
            batch_words,batch_tags = [],[]
            for sent in sentences[batch_indices]:
                words,tags = zip(*sent)
                batch_words.append(words)
                batch_tags.append(tags)

            batch_words = to_matrix(batch_words,word_to_id,max_len,pad)
            batch_tags = to_matrix(batch_tags,tag_to_id,max_len,pad)

            batch_tags_1hot = to_categorical(batch_tags,len(all_tags)).reshape(batch_tags.shape+(-1,))
            yield batch_words,batch_tags_1hot

In [11]:
def compute_test_accuracy(model):
    test_words,test_tags = zip(*[zip(*sentence) for sentence in test_data])
    test_words,test_tags = to_matrix(test_words,word_to_id),to_matrix(test_tags,tag_to_id)

    #predict tag probabilities of shape [batch,time,n_tags]
    predicted_tag_probabilities = model.predict(test_words,verbose=1)
    predicted_tags = predicted_tag_probabilities.argmax(axis=-1)

    #compute accurary excluding padding
    numerator = np.sum(np.logical_and((predicted_tags == test_tags),(test_words != 0)))
    denominator = np.sum(test_words != 0)
    return float(numerator)/denominator


class EvaluateAccuracy(keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs=None):
        sys.stdout.flush()
        print("\nMeasuring validation accuracy...")
        acc = compute_test_accuracy(self.model)
        print("\nValidation accuracy: %.5f\n"%acc)

        sys.stdout.flush()

In [12]:
# Собираем модель
model.compile('adam', 'categorical_crossentropy')

# Обучаем модель
model.fit(generate_batches(train_data), steps_per_epoch = (len(train_data) / BATCH_SIZE), epochs = 5)

Epoch 1/5
1343/1343 [==============================] - 9s 6ms/step - loss: 0.2279
Epoch 2/5
1343/1343 [==============================] - 9s 7ms/step - loss: 0.0575
Epoch 3/5
1343/1343 [==============================] - 9s 7ms/step - loss: 0.0508
Epoch 4/5
1343/1343 [==============================] - 9s 7ms/step - loss: 0.0463
Epoch 5/5
1343/1343 [==============================] - 10s 7ms/step - loss: 0.0425


In [14]:
# Проверяем достигнутую точность
acc = compute_test_accuracy(model)
print("Final accuracy: %.5f"%acc)

assert acc>0.94, "Keras has gone o9=-0367854 a rampage again, please contact course staff."

448/448 [==============================] - 2s 5ms/step
Final accuracy: 0.94627


#### Модель двунаправленной рекуррентной нейросети

In [72]:
# Нейросеть последовательной архитектуры
model1 = keras.models.Sequential()
# Входной слой
model1.add(L.InputLayer([None], dtype = 'int32'))
# Слой сжатия, количество входов равно количеству всех слов, выходов 50
model1.add(L.Embedding(len(all_words), 50))
# Слой двунаправленной рекррентной нейросети
model1.add(L.Bidirectional(L.SimpleRNN(64, return_sequences = True)))

# Выходной слой размерностью равной количеству всех слов выдаёт веростности для тегов
stepwise_dense = L.Dense(len(all_tags), activation = 'softmax')
stepwise_dense = L.TimeDistributed(stepwise_dense)
model1.add(stepwise_dense)

In [73]:
# Собираем модель
model1.compile('adam', 'categorical_crossentropy')

# Обучаем модель
model1.fit(generate_batches(train_data), steps_per_epoch = (len(train_data) / BATCH_SIZE), epochs = 5)

Epoch 1/5
1343/1343 [==============================] - 15s 10ms/step - loss: 0.1978
Epoch 2/5
1343/1343 [==============================] - 14s 10ms/step - loss: 0.0429
Epoch 3/5
1343/1343 [==============================] - 14s 10ms/step - loss: 0.0351
Epoch 4/5
1343/1343 [==============================] - 13s 10ms/step - loss: 0.0301
Epoch 5/5
1343/1343 [==============================] - 14s 10ms/step - loss: 0.0255


In [75]:
acc = compute_test_accuracy(model1)
print("\nFinal accuracy: %.5f"%acc)

assert acc>0.96, "Bidirectional RNNs are better than this!"
print("Well done!")

448/448 [==============================] - 4s 8ms/step

Final accuracy: 0.96125
Well done!
